In [9]:
%load_ext autoreload
%autoreload 2 
%load_ext autoreload
%autoreload 
#import sys
#for p in sys.path:
#    print(p)
!jupyter kernelspec list
#!pip install networkx
#!pip install node2vec
import numpy as np
import math as math 
import tensorflow as tf
import graph_synthesis as gs
from random_mini_batches import random_mini_batches_conv_perm
import networkx as nx
import node2vec as n2v
tf.test.gpu_device_name()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Available kernels:
  python2           /home/ubuntu/.local/share/jupyter/kernels/python2
  pyspark3kernel    /home/ubuntu/anaconda3/envs/python3/share/jupyter/kernels/pyspark3kernel
  pysparkkernel     /home/ubuntu/anaconda3/envs/python3/share/jupyter/kernels/pysparkkernel
  python3           /home/ubuntu/anaconda3/envs/python3/share/jupyter/kernels/python3
  sparkkernel       /home/ubuntu/anaconda3/envs/python3/share/jupyter/kernels/sparkkernel
  sparkrkernel      /home/ubuntu/anaconda3/envs/python3/share/jupyter/kernels/sparkrkernel


ModuleNotFoundError: No module named 'tensorflow'

In [5]:
class CNN:
    
    def __init__(self, inputdat, labeldat, testdat, testlabeldat, layer_dims, decayrate = 0.975,
                 l2reg = 1e-2, learning_rate = 0.008, batch_size = 128, epoch_nums = 8000):
        
        #Data
        self.inputdat = inputdat
        self.labeldat = labeldat
        self.testdat = testdat
        self.testlabeldat = testlabeldat
        self.num_eg = inputdat.shape[0]
        self.inputdim = inputdat.shape[1]
        
        #Hyperparameters
        self.layer_dims = layer_dims
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epoch_nums = epoch_nums
        self.l2reg = l2reg  
        self.decayrate = decayrate 
        
        #Initialize
        tf.reset_default_graph()
        self.initdata()
        
        #Create graph
        self.output = self.model()
        self.optimizer, self.cost = self.train()
        self.prediction, self.accuracy = self.test()
        self.summary_op = self.create_summaries()
        
        #Run Tensorflow
        config = tf.ConfigProto(allow_soft_placement = True, log_device_placement = True, device_count = {'GPU': 1})
        self.sess = tf.Session(config = config)
        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        self.sess.run(init)
        
        #Tensorboard setup
        self.writer = tf.summary.FileWriter('./CNNSave',  (self.sess).graph)
        
        #Train model 
        self.graph_classifier()
        self.prediction_num, self.error_num = self.evaluate("train") 
        
    def  create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar("loss", self.cost)
            #tf.summary.scalar("output", self.output)
            tf.summary.scalar("accuracy", self.accuracy)
            tf.summary.histogram("histogram_loss", self.cost)
            #tf.summary.histogram("histogram_loss", self.output)
            summary_op = tf.summary.merge_all()
        return summary_op
    
    def initdata(self):
        self.tflearnrate = tf.placeholder(tf.float32, name = "learnrate")
        self.mode = tf.placeholder(tf.bool, name = "mode")
        self.input = tf.placeholder(tf.float32, shape = [None, self.inputdim, self.inputdim, 1], name = "input")
        self.label = tf.placeholder(tf.float32, shape = [None, 1], name = "label")    
    
    def model(self):
        layer_dims = self.layer_dims
        layers = {}
        
        #Convolution 1 
        scope = 'convolayers' + str(1) 
        with tf.variable_scope(scope):
            conv11 = tf.layers.conv2d(inputs=self.input, filters=32, kernel_size=[3, 3],
                                     padding="same", activation=None)
            conv11 = tf.contrib.layers.batch_norm(conv11, is_training = self.mode, scope = 'batchnorm1')
            conv11 = tf.nn.relu(conv11, 'relu1')
            
            conv12 = tf.layers.conv2d(inputs=conv11, filters=32, kernel_size = [5, 5], 
                                     padding="same", activation=None)
            conv12 = tf.contrib.layers.batch_norm(conv12, is_training = self.mode, scope = 'batchnorm2')
            conv12 = tf.nn.relu(conv12, 'relu2')
            
            pool1 = conv12
            #pool1 = tf.layers.max_pooling2d(inputs=conv12, pool_size=[2, 2], strides=1)
        
        #Convolution 2 
        scope = 'convolayers' + str(2) 
        with tf.variable_scope(scope):
            conv21 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[3, 3],
                                     padding="same", activation=None)
            conv21 = tf.contrib.layers.batch_norm(conv21, is_training = self.mode, scope = 'batchnorm1')
            conv21 = tf.nn.relu(conv21, 'relu1')
            
            conv22 = tf.layers.conv2d(inputs=conv21, filters=64, kernel_size = [5, 5], 
                                     padding="same", activation=None)
            conv22 = tf.contrib.layers.batch_norm(conv22, is_training = self.mode, scope = 'batchnorm2')
            conv22 = tf.nn.relu(conv22, 'relu2')
            
            pool2 = tf.layers.max_pooling2d(inputs=conv22, pool_size=[2, 2], strides=2)
        
        #Convolution 3
        scope = 'convolayers' + str(3) 
        with tf.variable_scope(scope):
            conv31 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=[3, 3],
                                     padding="same", activation=None)
            conv31 = tf.contrib.layers.batch_norm(conv31, is_training = self.mode, scope = 'batchnorm1')
            conv31 = tf.nn.relu(conv31, 'relu1')
            
            conv32 = tf.layers.conv2d(inputs=conv31, filters=128, kernel_size = [3, 3], 
                                     padding="same", activation=None)
            conv32 = tf.contrib.layers.batch_norm(conv32, is_training = self.mode, scope = 'batchnorm2')
            conv32 = tf.nn.relu(conv32, 'relu2')
            
            #pool3 = conv32
            pool3 = tf.layers.max_pooling2d(inputs=conv32, pool_size=[2, 2], strides=2)
            
        size = pool3.shape[1]
        pool_flat = tf.reshape(pool3, [-1, size * size * 128])
    
        layers[0] = pool_flat
        #Dense layers
        for i in range(len(layer_dims)-1):
            if (i < len(layer_dims) - 2):
                scope = 'denselayers' + str(i+1)
                with tf.variable_scope(scope):
                    layers[i+1] = tf.contrib.layers.fully_connected(num_outputs = layer_dims[i+1], \
                                                        activation_fn = None, inputs = layers[i], scope = 'dense')
                    layers[i+1] = tf.contrib.layers.batch_norm(layers[i+1], is_training = self.mode, scope = 'batchnorm')
                    layers[i+1] = tf.nn.relu(layers[i+1], 'relu')
            else:
                scope = 'denselayers' + str(i+1)
                with tf.variable_scope(scope):
                    layers[len(layer_dims)-1] = tf.layers.dense(units = layer_dims[len(layer_dims)-1], \
                                                    activation = tf.nn.sigmoid, inputs = layers[len(layer_dims)-2])
        
        output = layers[len(layer_dims)-1]
        return output 
    
    def train(self):
        #Saver for data 
        self.saver = tf.train.Saver()
        self.global_step = tf.Variable(0, dtype = tf.int32, trainable = False, name = 'global_step')
        
        #Define cost 
        gap_loss = tf.reduce_max(self.output) - tf.reduce_min(self.output)
        reg_loss = self.l2reg * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()) 
        cost = (tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.output, labels = self.label)) + \
                                                                                                            reg_loss)
        #cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.output, labels = self.label)) * 1/(0.01+gap_loss) + reg_loss
        
        #Batch Normalization
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.tflearnrate).minimize(cost, global_step = self.global_step)
            
        return optimizer, cost 
    
    def test(self):
        prediction = tf.greater(self.output, tf.constant(0.5, dtype = tf.float32))
        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.cast(prediction, dtype = tf.float32), self.label)
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                #tf.summary.scalar('accuracy', accuracy)
        return prediction, accuracy
    
    def graph_classifier(self):
        curlearnrate = self.learning_rate
        step = 0 
        for epochs in range(self.epoch_nums):
            output = []
            mini_batch = random_mini_batches_conv_perm(self.inputdat, self.labeldat, self.batch_size)
            curlearnrate = curlearnrate * self.decayrate 
            for dat in mini_batch:
                step = step + 1 
                _, cost, output, summary = self.sess.run([self.optimizer, self.cost, self.output, self.summary_op], 
                                                feed_dict = {self.input: dat[0], self.label: dat[1], \
                                                             self.mode: True, self.tflearnrate: curlearnrate})
                (self.writer).add_summary(summary, global_step = step)
            if ((epochs+1) % 50 == 0):
                
                _, errortrain = self.evaluate("train")
                output, cost, accuracy = self.sess.run([self.output, self.cost, self.accuracy], \
                        feed_dict = {self.input: self.inputdat, self.label: self.labeldat, \
                                     self.mode: False, self.tflearnrate: curlearnrate})
                (self.saver).save(self.sess, 'checkpoints/CNN', global_step=self.global_step)
                _, errortest = self.evaluate("test")
                print("epoch", epochs + 1, "    |    ", "cost", "%.6e" % cost, "    |    ", 
                      "error train", "%.3f" % (errortrain*100), "    |    ", "error test", "%.3f" % (errortest*100), 
                      "   |   ", "output gap", "%.6e" % (np.max(output) - np.min(output)))
            
                #if ((epochs+1) % 300 == 0):
                #    print(output)
    def evaluate(self, mode):
        #Get predictions
        if mode == "train":
            predictions = self.sess.run([self.prediction], 
                                    feed_dict = {self.input: self.inputdat, self.label: self.labeldat, \
                                                 self.mode: False})
        else:
            predictions = self.sess.run([self.prediction], 
                                    feed_dict = {self.input: self.testdat, self.label: self.testlabeldat, \
                                                 self.mode: False})
        predictions = np.array(predictions).reshape((self.num_eg,1))
        predictions = predictions.astype(int)
        
        #Compute error
        error = np.sum((predictions - self.labeldat)**2)/self.num_eg
        #print("error " + mode, error * 100)
        
        return predictions, error

In [6]:
if  __name__ == "__main__":
    
    num_eg = 1000
    vertex = 16
    inputdim = vertex * vertex
    
    #train ensemble 
    data = gs.generate_ensemble([int(num_eg/2), int(num_eg/2)], vertex, [1/3, 1/2])
    #data = gs.generate_ensemble_v2(num_eg, vertex)
    #test ensemble
    testdata = gs.generate_ensemble([int(num_eg/2), int(num_eg/2)], vertex, [1/3, 1/2])
    #testdata = gs.generate_ensemble_v2(num_eg, vertex)
    #reformat 
    data["graphs"] = data["graphs"].reshape((num_eg, vertex, vertex, 1))
    testdata["graphs"] = testdata["graphs"].reshape((num_eg, vertex, vertex, 1))
    
    #Normalize
    meandata = np.mean(data["graphs"], axis = 0)
    stddata = np.std(data["graphs"], axis = 0)
    data["graphs"] = np.divide((data["graphs"] - meandata), stddata)
    
    meantestdata = np.mean(testdata["graphs"], axis = 0)
    stdtestdata = np.std(testdata["graphs"], axis = 0)
    testdata["graphs"] = np.divide((testdata["graphs"] - meantestdata), stdtestdata)
    
    #Fully connected model 
    CNN = CNN(data["graphs"], data["labels"], testdata["graphs"], testdata["labels"], [inputdim,350,100,50,1])
    

NameError: name 'gs' is not defined